In [51]:
import pandas as pd
import nltk
import re
import string
from nltk.corpus import stopwords
import preprocessor as p
import wordninja

In [33]:
data = pd.read_csv('../../Data/new_data.csv',index_col=False)

In [34]:
data.head()

,Tweet Text,Informativeness
0,RT @mariexplorer: Good morning #cebu #sea #par...,0
1,Where is Liloan Central Field and how do we ge...,0
2,"Earthquake in Iloilo, Philippines! My head's a...",0
3,RT @LastQuake: NEW: Felt #earthquake M7.0 - Bo...,1
4,"RT @NewEarthquake: 7.2 earthquake, 2km S of Nu...",1


In [11]:
def to_lower(text):
    return text.lower()

In [15]:
def remove_punctuations(text):
    return re.sub('[%s]' % re.escape(string.punctuation),'',text)


'Earthquake in Iloilo Philippines My heads aching'

In [19]:
stop_words = set(stopwords.words('english'))
stop_words.add('http')
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])


In [79]:
def rephrase_text(text):
    p.set_options(p.OPT.URL,p.OPT.MENTION)
    parsed_text = p.tokenize(text)
    clean_text = re.sub("RT\s\$MENTION\$:","retweet",parsed_text)
    clean_text = re.sub("\$MENTION\$:","mentions",clean_text)
    clean_text = re.sub("\$URL\$","link",clean_text)
    return clean_text
    
test = data["Tweet Text"].iloc[91]
rephrase_text(test)

RT @ABC7: PHILIPPINES #EARTHQUAKE: Radio reports say 7.2-mag. quake has killed at least 4 people in central (more) http://t.co/MHGv5ifvHZ
RT $MENTION$: PHILIPPINES #EARTHQUAKE: Radio reports say 7.2-mag. quake has killed at least 4 people in central (more) $URL$
retweet PHILIPPINES #EARTHQUAKE: Radio reports say 7.2-mag. quake has killed at least 4 people in central (more) link


In [99]:
def handle_hasttag(text):
    p.set_options(p.OPT.HASHTAG) 
    parsed_text = p.parse(text)
    for hashtag in parsed_text.hashtags:
        match = hashtag.match
        text = re.sub(match, " ".join(wordninja.split(match)), text)
    return text

test = data["Tweet Text"].iloc[93]
handle_hasttag(test)

#PrayforBoholandCebu #PrayForBohol #PrayForCarmenBohol 🙏🙏
Pray for Bohol and Cebu Pray For Bohol Pray For Carmen Bohol 🙏🙏


In [104]:
def remove_emoji(text):
    p.set_options(p.OPT.EMOJI,p.OPT.SMILEY)
    tokenized_text = p.tokenize(text)
    tokenized_text = re.sub(re.escape("$EMOJI$"),"",tokenized_text)
    return tokenized_text


test = data["Tweet Text"].iloc[93]
remove_emoji(test)

#PrayforBoholandCebu #PrayForBohol #PrayForCarmenBohol $EMOJI$$EMOJI$
#PrayforBoholandCebu #PrayForBohol #PrayForCarmenBohol 
